In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import numpy as np
import dpsimpy
import re

# %matplotlib widget

epsilon = 1e-12

## Voltage Source + Resistor

In [ ]:
time_step = 0.00005
final_time = 1

In [ ]:
sim_name = "EMT_VoltageSource_Resistor"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

# Nodes
gnd = dpsimpy.emt.SimNode.gnd
n1 = dpsimpy.emt.SimNode("n1", dpsimpy.PhaseType.ABC)

# Components
vs = dpsimpy.emt.ph3.VoltageSource("vs1")
vs.set_parameters(
    dpsimpy.Math.single_phase_variable_to_three_phase(complex(100000, 0)), 50
)

res = dpsimpy.emt.ph3.Resistor("R1", dpsimpy.LogLevel.debug)
res.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(100))

# Topology
vs.connect([gnd, n1])
res.connect([n1, gnd])

sys = dpsimpy.SystemTopology(50, [n1], [vs, res])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("iR", "i_intf", res)

# Simulation
sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.add_logger(logger)
sim.run()

In [ ]:
sim_name = "DP_VoltageSource_Resistor"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

# Nodes
gnd = dpsimpy.dp.SimNode.gnd
n1 = dpsimpy.dp.SimNode("n1")

# Components
vs = dpsimpy.dp.ph1.VoltageSource("vs1")
vs.set_parameters(complex(100000, 0))

res = dpsimpy.dp.ph1.Resistor("R1", dpsimpy.LogLevel.debug)
res.set_parameters(100)

# Topology
vs.connect([gnd, n1])
res.connect([n1, gnd])

sys = dpsimpy.SystemTopology(50, [n1], [vs, res])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("iR", "i_intf", res)

# Simulation
sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.DP)
sim.add_logger(logger)
sim.run()

In [ ]:
sim_name = "SP_VoltageSource_Resistor"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

# Nodes
gnd = dpsimpy.sp.SimNode.gnd
n1 = dpsimpy.sp.SimNode("n1")

# Components
vs = dpsimpy.sp.ph1.VoltageSource("vs1")
vs.set_parameters(complex(100000, 0))

res = dpsimpy.sp.ph1.Resistor("R1", dpsimpy.LogLevel.debug)
res.set_parameters(100)

# Topology
vs.connect([gnd, n1])
res.connect([n1, gnd])

sys = dpsimpy.SystemTopology(50, [n1], [vs, res])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("iR", "i_intf", res)

# Simulation
sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.SP)
sim.add_logger(logger)
sim.run()

In [ ]:
model_name = "VoltageSource_Resistor"

path_SP = "logs/" + "SP_" + model_name + "/"
dpsim_result_file_SP = path_SP + "SP_" + model_name + ".csv"
ts_dpsim_SP = read_timeseries_csv(dpsim_result_file_SP)
ts_dpsim_SP_shifted = ts.frequency_shift_list(ts_dpsim_SP, 50)

path_DP = "logs/" + "DP_" + model_name + "/"
dpsim_result_file_DP = path_DP + "DP_" + model_name + ".csv"
ts_dpsim_DP = read_timeseries_csv(dpsim_result_file_DP)
ts_dpsim_DP_shifted = ts.frequency_shift_list(ts_dpsim_DP, 50)

path_EMT = "logs/" + "EMT_" + model_name + "/"
dpsim_result_file_EMT = path_EMT + "EMT_" + model_name + ".csv"
ts_dpsim_EMT = read_timeseries_csv(dpsim_result_file_EMT)

### Plot voltages

In [ ]:
plt.figure()

# EMT
var_names = ["v1_0"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_EMT[var_name].time,
        np.sqrt(3 / 2) * ts_dpsim_EMT[var_name].values,
        label=var_name,
    )

# DP
var_names = ["v1_shift"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_DP_shifted[var_name].time,
        ts_dpsim_DP_shifted[var_name].values,
        label=var_name + " (DP)",
        linestyle="-.",
    )

# SP
var_names = ["v1_shift"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_SP_shifted[var_name].time,
        ts_dpsim_SP_shifted[var_name].values,
        label=var_name + " (SP)",
        linestyle=":",
    )

plt.legend()
plt.show()

### Plot currents

In [ ]:
plt.figure()

# EMT
var_names = ["iR_0"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_EMT[var_name].time,
        np.sqrt(3 / 2) * ts_dpsim_EMT[var_name].values,
        label=var_name,
    )

# DP
var_names = ["iR_shift"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_DP_shifted[var_name].time,
        ts_dpsim_DP_shifted[var_name].values,
        label=var_name + " (DP)",
        linestyle="-.",
    )

# SP
var_names = ["iR_shift"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_SP_shifted[var_name].time,
        ts_dpsim_SP_shifted[var_name].values,
        label=var_name + " (SP)",
        linestyle=":",
    )

plt.legend()
plt.show()

### Comparison SP vs. DP

In [ ]:
plt.figure()
for name in [("v1_shift", "v1_shift")]:
    plt.plot(
        ts_dpsim_SP_shifted[name[0]].time,
        ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values,
        label=name[0] + " (SP) vs. " + name[1] + " (DP)",
    )
plt.legend()
plt.show()

In [ ]:
plt.figure()
for name in [("iR_shift", "iR_shift")]:
    plt.plot(
        ts_dpsim_SP_shifted[name[0]].time,
        ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values,
        label=name[0] + " (SP) vs. " + name[1] + " (DP)",
    )
plt.legend()
plt.show()

### Assertion SP vs. DP

In [ ]:
compare_errors_abs = []
compare_errors_rel = []
for name in [("v1_shift", "v1_shift"), ("iR_shift", "iR_shift")]:
    compare_errors_abs.append(
        np.absolute(
            ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values
        ).max()
    )
    compare_errors_rel.append(
        np.absolute(
            ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values
        ).max()
        / ts_dpsim_DP_shifted[name[1]].values.max()
    )
    print(name[0] + " vs. " + name[1] + " (abs): " + str(compare_errors_abs[-1]))
    print(name[0] + " vs. " + name[1] + " (rel): " + str(compare_errors_rel[-1]))
print("Max rel error: " + "{:.2}".format(np.max(compare_errors_rel) * 100) + "%")
assert np.max(compare_errors_rel) < epsilon

### Comparison DP vs. EMT

In [ ]:
plt.figure()
for name in [("v1_0", "v1_shift")]:
    plt.plot(
        ts_dpsim_EMT[name[0]].time,
        np.sqrt(3 / 2) * ts_dpsim_EMT[name[0]].values
        - ts_dpsim_DP_shifted[name[1]].values,
        label=name[0] + " vs. " + name[1],
    )
plt.legend()
plt.show()

In [ ]:
plt.figure()
for name in [("iR_0", "iR_shift")]:
    plt.plot(
        ts_dpsim_EMT[name[0]].time,
        np.sqrt(3 / 2) * ts_dpsim_EMT[name[0]].values
        - ts_dpsim_DP_shifted[name[1]].values,
        label=name[0] + " vs. " + name[1],
    )
plt.legend()
plt.show()

### Assertion DP vs. EMT

In [ ]:
compare_errors_abs = []
compare_errors_rel = []
for name in [("v1_0", "v1_shift"), ("iR_0", "iR_shift")]:
    compare_errors_abs.append(
        np.absolute(
            np.sqrt(3 / 2) * ts_dpsim_EMT[name[0]].values
            - ts_dpsim_DP_shifted[name[1]].values
        ).max()
    )
    compare_errors_rel.append(
        np.absolute(
            np.sqrt(3 / 2) * ts_dpsim_EMT[name[0]].values
            - ts_dpsim_DP_shifted[name[1]].values
        ).max()
        / ts_dpsim_DP_shifted[name[1]].values.max()
    )
    print(name[0] + " vs. " + name[1] + " (abs): " + str(compare_errors_abs[-1]))
    print(name[0] + " vs. " + name[1] + " (rel): " + str(compare_errors_rel[-1]))
print("Max rel error: " + "{:.2}".format(np.max(compare_errors_rel) * 100) + "%")
assert np.max(compare_errors_rel) < 1e-3

## Voltage Source + Resistor + Inductor

In [ ]:
time_step = 0.00005
final_time = 1

In [ ]:
sim_name = "EMT_VoltageSource_Inductor"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

# Nodes
gnd = dpsimpy.emt.SimNode.gnd
n1 = dpsimpy.emt.SimNode("n1", dpsimpy.PhaseType.ABC)
n2 = dpsimpy.emt.SimNode("n2", dpsimpy.PhaseType.ABC)

# Components
vs = dpsimpy.emt.ph3.VoltageSource("vs1", dpsimpy.LogLevel.debug)
vs.set_parameters(
    dpsimpy.Math.single_phase_variable_to_three_phase(complex(100000, 0)), 50
)

res = dpsimpy.emt.ph3.Resistor("R1", dpsimpy.LogLevel.debug)
res.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(5))

ind = dpsimpy.emt.ph3.Inductor("L1", dpsimpy.LogLevel.debug)
ind.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(0.5))

# Topology
vs.connect([gnd, n1])
res.connect([n1, n2])
ind.connect([n2, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, res, ind])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("iR", "i_intf", res)
logger.log_attribute("iL", "i_intf", ind)

# Simulation
sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.add_logger(logger)
sim.run()

In [ ]:
sim_name = "DP_VoltageSource_Inductor"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

# Nodes
gnd = dpsimpy.dp.SimNode.gnd
n1 = dpsimpy.dp.SimNode("n1")
n2 = dpsimpy.dp.SimNode("n2")

# Components
vs = dpsimpy.dp.ph1.VoltageSource("vs1", dpsimpy.LogLevel.debug)
vs.set_parameters(complex(100000, 0))

res = dpsimpy.dp.ph1.Resistor("R1", dpsimpy.LogLevel.debug)
res.set_parameters(5)

ind = dpsimpy.dp.ph1.Inductor("L1", dpsimpy.LogLevel.debug)
ind.set_parameters(0.5)

# Topology
vs.connect([gnd, n1])
res.connect([n1, n2])
ind.connect([n2, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, res, ind])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("iR", "i_intf", res)
logger.log_attribute("iL", "i_intf", ind)

# Simulation
sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.DP)
sim.add_logger(logger)
sim.run()

In [ ]:
sim_name = "SP_VoltageSource_Inductor"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

# Nodes
gnd = dpsimpy.sp.SimNode.gnd
n1 = dpsimpy.sp.SimNode("n1")
n2 = dpsimpy.sp.SimNode("n2")

# Components
vs = dpsimpy.sp.ph1.VoltageSource("vs1", dpsimpy.LogLevel.debug)
vs.set_parameters(complex(100000, 0))

res = dpsimpy.sp.ph1.Resistor("R1", dpsimpy.LogLevel.debug)
res.set_parameters(5)

ind = dpsimpy.sp.ph1.Inductor("L1", dpsimpy.LogLevel.debug)
ind.set_parameters(0.5)

# Topology
vs.connect([gnd, n1])
res.connect([n1, n2])
ind.connect([n2, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, res, ind])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("iR", "i_intf", res)
logger.log_attribute("iL", "i_intf", ind)

# Simulation
sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.SP)
sim.add_logger(logger)
sim.run()

In [ ]:
model_name = "VoltageSource_Inductor"

path_SP = "logs/" + "SP_" + model_name + "/"
dpsim_result_file_SP = path_SP + "SP_" + model_name + ".csv"
ts_dpsim_SP = read_timeseries_csv(dpsim_result_file_SP)
ts_dpsim_SP_shifted = ts.frequency_shift_list(ts_dpsim_SP, 50)

path_DP = "logs/" + "DP_" + model_name + "/"
dpsim_result_file_DP = path_DP + "DP_" + model_name + ".csv"
ts_dpsim_DP = read_timeseries_csv(dpsim_result_file_DP)
ts_dpsim_DP_shifted = ts.frequency_shift_list(ts_dpsim_DP, 50)

path_EMT = "logs/" + "EMT_" + model_name + "/"
dpsim_result_file_EMT = path_EMT + "EMT_" + model_name + ".csv"
ts_dpsim_EMT = read_timeseries_csv(dpsim_result_file_EMT)

### Plot voltages

In [ ]:
plt.figure()

# EMT
var_names = ["v1_0", "v2_0"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_EMT[var_name].time,
        np.sqrt(3 / 2) * ts_dpsim_EMT[var_name].values,
        label=var_name,
    )

# DP
var_names = ["v1_shift", "v2_shift"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_DP_shifted[var_name].time,
        ts_dpsim_DP_shifted[var_name].values,
        label=var_name + " (DP)",
        linestyle="-.",
    )

# SP
var_names = ["v1_shift", "v2_shift"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_SP_shifted[var_name].time,
        ts_dpsim_SP_shifted[var_name].values,
        label=var_name + " (SP)",
        linestyle=":",
    )

plt.legend()
plt.show()

### Plot currents

In [ ]:
plt.figure()

# EMT
var_names = ["iR_0", "iL_0"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_EMT[var_name].time,
        np.sqrt(3 / 2) * ts_dpsim_EMT[var_name].values,
        label=var_name,
    )

# DP
var_names = ["iR_shift", "iL_shift"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_DP_shifted[var_name].time,
        ts_dpsim_DP_shifted[var_name].values,
        label=var_name + " (DP)",
        linestyle=":",
    )

# SP
var_names = ["iR_shift", "iL_shift"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_SP_shifted[var_name].time,
        ts_dpsim_SP_shifted[var_name].values,
        label=var_name + " (SP)",
        linestyle=":",
    )

plt.legend()
plt.show()

### Comparison SP vs. DP

In [ ]:
plt.figure()
for name in [("v1_shift", "v1_shift"), ("v2_shift", "v2_shift")]:
    plt.plot(
        ts_dpsim_SP_shifted[name[0]].time,
        ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values,
        label=name[0] + " (SP) vs. " + name[1] + " (DP)",
    )
plt.legend()
plt.show()

In [ ]:
plt.figure()
for name in [("iR_shift", "iR_shift"), ("iL_shift", "iL_shift")]:
    plt.plot(
        ts_dpsim_SP_shifted[name[0]].time,
        ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values,
        label=name[0] + " (SP) vs. " + name[1] + " (DP)",
    )
plt.legend()
plt.show()

## Assertion SP vs. DP

In [ ]:
compare_errors_abs = []
compare_errors_rel = []
for name in [
    ("v1_shift", "v1_shift"),
    ("v2_shift", "v2_shift"),
    ("iR_shift", "iR_shift"),
    ("iL_shift", "iL_shift"),
]:
    compare_errors_abs.append(
        np.absolute(
            ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values
        ).max()
    )
    compare_errors_rel.append(
        np.absolute(
            ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values
        ).max()
        / ts_dpsim_DP_shifted[name[1]].values.max()
    )
    print(name[0] + " vs. " + name[1] + " (abs): " + str(compare_errors_abs[-1]))
    print(name[0] + " vs. " + name[1] + " (rel): " + str(compare_errors_rel[-1]))
print("Max rel error: " + "{:.2}".format(np.max(compare_errors_rel) * 100) + "%")
assert np.max(compare_errors_rel) < 3e-1

### Comparison DP vs. EMT

In [ ]:
plt.figure()
for name in [("v1_0", "v1_shift"), ("v2_0", "v2_shift")]:
    plt.plot(
        ts_dpsim_EMT[name[0]].time,
        np.sqrt(3 / 2) * ts_dpsim_EMT[name[0]].values
        - ts_dpsim_DP_shifted[name[1]].values,
        label=name[0] + " vs. " + name[1],
    )
plt.legend()
plt.show()

In [ ]:
plt.figure()
for name in [("iR_0", "iR_shift"), ("iL_0", "iL_shift")]:
    plt.plot(
        ts_dpsim_EMT[name[0]].time,
        np.sqrt(3 / 2) * ts_dpsim_EMT[name[0]].values
        - ts_dpsim_DP_shifted[name[1]].values,
        label=name[0] + " vs. " + name[1],
    )
plt.legend()
plt.show()

### Assertion DP vs. EMT

In [ ]:
compare_errors_abs = []
compare_errors_rel = []
for name in [
    ("v1_0", "v1_shift"),
    ("v2_0", "v2_shift"),
    ("iR_0", "iR_shift"),
    ("iL_0", "iL_shift"),
]:
    compare_errors_abs.append(
        np.absolute(
            np.sqrt(3 / 2) * ts_dpsim_EMT[name[0]].values
            - ts_dpsim_DP_shifted[name[1]].values
        ).max()
    )
    compare_errors_rel.append(
        np.absolute(
            np.sqrt(3 / 2) * ts_dpsim_EMT[name[0]].values
            - ts_dpsim_DP_shifted[name[1]].values
        ).max()
        / ts_dpsim_DP_shifted[name[1]].values.max()
    )
    print(name[0] + " vs. " + name[1] + " (abs): " + str(compare_errors_abs[-1]))
    print(name[0] + " vs. " + name[1] + " (rel): " + str(compare_errors_rel[-1]))
print("Max rel error: " + "{:.2}".format(np.max(compare_errors_rel) * 100) + "%")
assert np.max(compare_errors_rel) < 1e-3

## Voltage Source + Resistor + Capacitor

In [ ]:
time_step = 0.00005
final_time = 1

In [ ]:
sim_name = "EMT_VoltageSource_Capacitor"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

# Nodes
gnd = dpsimpy.emt.SimNode.gnd
n1 = dpsimpy.emt.SimNode("n1", dpsimpy.PhaseType.ABC)
n2 = dpsimpy.emt.SimNode("n2", dpsimpy.PhaseType.ABC)

# Components
vs = dpsimpy.emt.ph3.VoltageSource("vs1", dpsimpy.LogLevel.debug)
vs.set_parameters(
    dpsimpy.Math.single_phase_variable_to_three_phase(complex(100000, 0)), 50
)

res = dpsimpy.emt.ph3.Resistor("R1", dpsimpy.LogLevel.debug)
res.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(5))

cap = dpsimpy.emt.ph3.Capacitor("C1", dpsimpy.LogLevel.debug)
cap.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(10e-3))

# Topology
vs.connect([gnd, n1])
res.connect([n1, n2])
cap.connect([n2, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, res, cap])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("iR", "i_intf", res)
logger.log_attribute("iC", "i_intf", cap)

# Simulation
sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.add_logger(logger)
sim.run()

In [ ]:
sim_name = "DP_VoltageSource_Capacitor"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

# Nodes
gnd = dpsimpy.dp.SimNode.gnd
n1 = dpsimpy.dp.SimNode("n1")
n2 = dpsimpy.dp.SimNode("n2")

# Components
vs = dpsimpy.dp.ph1.VoltageSource("vs1", dpsimpy.LogLevel.debug)
vs.set_parameters(complex(100000, 0))

res = dpsimpy.dp.ph1.Resistor("R1", dpsimpy.LogLevel.debug)
res.set_parameters(5)

cap = dpsimpy.dp.ph1.Capacitor("C1", dpsimpy.LogLevel.debug)
cap.set_parameters(10e-3)

# Topology
vs.connect([gnd, n1])
res.connect([n1, n2])
cap.connect([n2, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, res, cap])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("iR", "i_intf", res)
logger.log_attribute("iC", "i_intf", cap)

# Simulation
sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.DP)
sim.add_logger(logger)
sim.run()

In [ ]:
sim_name = "SP_VoltageSource_Capacitor"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

# Nodes
gnd = dpsimpy.sp.SimNode.gnd
n1 = dpsimpy.sp.SimNode("n1")
n2 = dpsimpy.sp.SimNode("n2")

# Components
vs = dpsimpy.sp.ph1.VoltageSource("vs1", dpsimpy.LogLevel.debug)
vs.set_parameters(complex(100000, 0))

res = dpsimpy.sp.ph1.Resistor("R1", dpsimpy.LogLevel.debug)
res.set_parameters(5)

cap = dpsimpy.sp.ph1.Capacitor("C1", dpsimpy.LogLevel.debug)
cap.set_parameters(10e-3)

# Topology
vs.connect([gnd, n1])
res.connect([n1, n2])
cap.connect([n2, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, res, cap])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("iR", "i_intf", res)
logger.log_attribute("iC", "i_intf", cap)

# Simulation
sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.SP)
sim.add_logger(logger)
sim.run()

In [ ]:
model_name = "VoltageSource_Capacitor"

path_SP = "logs/" + "SP_" + model_name + "/"
dpsim_result_file_SP = path_SP + "SP_" + model_name + ".csv"
ts_dpsim_SP = read_timeseries_csv(dpsim_result_file_SP)
ts_dpsim_SP_shifted = ts.frequency_shift_list(ts_dpsim_SP, 50)

path_DP = "logs/" + "DP_" + model_name + "/"
dpsim_result_file_DP = path_DP + "DP_" + model_name + ".csv"
ts_dpsim_DP = read_timeseries_csv(dpsim_result_file_DP)
ts_dpsim_DP_shifted = ts.frequency_shift_list(ts_dpsim_DP, 50)

path_EMT = "logs/" + "EMT_" + model_name + "/"
dpsim_result_file_EMT = path_EMT + "EMT_" + model_name + ".csv"
ts_dpsim_EMT = read_timeseries_csv(dpsim_result_file_EMT)

### Plot voltages

In [ ]:
plt.figure()

# EMT
var_names = ["v1_0", "v2_0"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_EMT[var_name].time,
        np.sqrt(3 / 2) * ts_dpsim_EMT[var_name].values,
        label=var_name,
    )

# DP
var_names = ["v1_shift", "v2_shift"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_DP_shifted[var_name].time,
        ts_dpsim_DP_shifted[var_name].values,
        label=var_name + " (DP)",
        linestyle="-.",
    )

# SP
var_names = ["v1_shift", "v2_shift"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_SP_shifted[var_name].time,
        ts_dpsim_SP_shifted[var_name].values,
        label=var_name + " (SP)",
        linestyle=":",
    )

plt.legend()
plt.show()

### Plot currents

In [ ]:
plt.figure()

# EMT
var_names = ["iR_0", "iC_0"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_EMT[var_name].time,
        np.sqrt(3 / 2) * ts_dpsim_EMT[var_name].values,
        label=var_name,
    )

# DP
var_names = ["iR_shift", "iC_shift"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_DP_shifted[var_name].time,
        ts_dpsim_DP_shifted[var_name].values,
        label=var_name + " (DP)",
        linestyle=":",
    )

# SP
var_names = ["iR_shift", "iC_shift"]
for var_name in var_names:
    plt.plot(
        ts_dpsim_DP_shifted[var_name].time,
        ts_dpsim_DP_shifted[var_name].values,
        label=var_name + " (SP)",
        linestyle=":",
    )

plt.legend()
plt.show()

### Comparison SP vs. DP

In [ ]:
plt.figure()
for name in [("v1_shift", "v1_shift"), ("v2_shift", "v2_shift")]:
    plt.plot(
        ts_dpsim_SP_shifted[name[0]].time,
        ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values,
        label=name[0] + " (SP) vs. " + name[1] + " (DP)",
    )
plt.legend()
plt.show()

In [ ]:
plt.figure()
for name in [("iR_shift", "iR_shift"), ("iC_shift", "iC_shift")]:
    plt.plot(
        ts_dpsim_SP_shifted[name[0]].time,
        ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values,
        label=name[0] + " (SP) vs. " + name[1] + " (DP)",
    )
plt.legend()
plt.show()

### Assertion SP vs. DP

In [ ]:
compare_errors_abs = []
compare_errors_rel = []
for name in [
    ("v1_shift", "v1_shift"),
    ("v2_shift", "v2_shift"),
    ("iR_shift", "iR_shift"),
    ("iC_shift", "iC_shift"),
]:
    compare_errors_abs.append(
        np.absolute(
            ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values
        ).max()
    )
    compare_errors_rel.append(
        np.absolute(
            ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values
        ).max()
        / ts_dpsim_DP_shifted[name[1]].values.max()
    )
    print(name[0] + " vs. " + name[1] + " (abs): " + str(compare_errors_abs[-1]))
    print(name[0] + " vs. " + name[1] + " (rel): " + str(compare_errors_rel[-1]))
print("Max rel error: " + "{:.2}".format(np.max(compare_errors_rel) * 100) + "%")
assert np.max(compare_errors_rel) < 6e-1

### Comparison DP vs. EMT

In [ ]:
plt.figure()
for name in [("v1_0", "v1_shift"), ("v2_0", "v2_shift")]:
    plt.plot(
        ts_dpsim_EMT[name[0]].time,
        np.sqrt(3 / 2) * ts_dpsim_EMT[name[0]].values
        - ts_dpsim_DP_shifted[name[1]].values,
        label=name[0] + " vs. " + name[1],
    )
plt.legend()
plt.show()

In [ ]:
plt.figure()
for name in [("iR_0", "iR_shift"), ("iC_0", "iC_shift")]:
    plt.plot(
        ts_dpsim_EMT[name[0]].time,
        np.sqrt(3 / 2) * ts_dpsim_EMT[name[0]].values
        - ts_dpsim_DP_shifted[name[1]].values,
        label=name[0] + " vs. " + name[1],
    )
plt.legend()
plt.show()

### Assertion DP vs. EMT

In [ ]:
compare_errors_abs = []
compare_errors_rel = []
for name in [
    ("v1_0", "v1_shift"),
    ("v2_0", "v2_shift"),
    ("iR_0", "iR_shift"),
    ("iC_0", "iC_shift"),
]:
    compare_errors_abs.append(
        np.absolute(
            np.sqrt(3 / 2) * ts_dpsim_EMT[name[0]].values
            - ts_dpsim_DP_shifted[name[1]].values
        ).max()
    )
    compare_errors_rel.append(
        np.absolute(
            np.sqrt(3 / 2) * ts_dpsim_EMT[name[0]].values
            - ts_dpsim_DP_shifted[name[1]].values
        ).max()
        / ts_dpsim_DP_shifted[name[1]].values.max()
    )
    print(name[0] + " vs. " + name[1] + " (abs): " + str(compare_errors_abs[-1]))
    print(name[0] + " vs. " + name[1] + " (rel): " + str(compare_errors_rel[-1]))
print("Max rel error: " + "{:.2}".format(np.max(compare_errors_rel) * 100) + "%")
assert np.max(compare_errors_rel) < 1e-3

### Compare frequency spectra

In [ ]:
# freq_dpsim_EMT = {}
# freq_dpsim_DP_shifted = {}
# freq_dpsim_SP_shifted = {}

# freq_dpsim_EMT['v2_0'] = np.fft.fft(ts_dpsim_EMT['v2_0'].values)
# freq_dpsim_DP_shifted['v2_shift'] = np.fft.fft(ts_dpsim_DP_shifted['v2_shift'].values)
# freq_dpsim_SP_shifted['v2_shift'] = np.fft.fft(ts_dpsim_SP_shifted['v2_shift'].values)

# plt.plot(freq_dpsim_DP_shifted['v2_shift'] - freq_dpsim_EMT['v2_0'], label='F[v2_shift] (DP) vs. F[v2_0]')
# plt.plot(freq_dpsim_DP_shifted['v2_shift'] - freq_dpsim_SP_shifted['v2_shift'], label='F[v2_shift] (DP) vs. F[v2_shift] (SP)')
# plt.plot(freq_dpsim_EMT['v2_0'])
# plt.plot(freq_dpsim_DP_shifted['v2_shift'])
# plt.plot(freq_dpsim_SP_shifted['v2_shift'])
plt.magnitude_spectrum(ts_dpsim_EMT["v2_0"].values, Fs=1 / 0.00005, label="F[v2_0]")
plt.magnitude_spectrum(
    ts_dpsim_DP_shifted["v2_shift"].values, Fs=1 / 0.00005, label="F[v2_shift] (DP)"
)
plt.magnitude_spectrum(
    ts_dpsim_SP_shifted["v2_shift"].values,
    Fs=1 / 0.00005,
    linestyle=":",
    label="F[v2_shift] (SP)",
)
# plt.gca().set_xlim([40, 60])
plt.legend()
plt.show()